## <b><center>ПРАКТИКА</center></b>



# Подготовка окружения для выполнения задания
В ходе выполнения задний нам понадобятся файлы с Google диска.
Для этого нам нужно подключить диск где расположены файлы для выполнения задания:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### <b> <center>4.  Синтез изображений</center></b>


&emsp;&emsp;Не всегда имеется возможность собрать большой датасет, особенно остро это касается узкоспециализированных задач, например, в задаче детекции спецтранспорта не всегда можно найти вариативный датасет в открытых источниках, а собирать такой датасет и размечать достаточно трудоемко.<br>
&emsp;&emsp;В таких задачах можно синтезировать датасет с разметкой, обучить на большом таком датасете, а затем дообучать на реальных данных.<br>
&emsp;&emsp;Существует ряд подходов:<br>
&emsp;&emsp;&emsp;&emsp;<b> - Подход на основе обучения модели генеративно-состязательной сети (GAN), и применения при генератора для генерации синтетических изображений </b><br>
<center><img src="https://drive.google.com/uc?id=1Tec7P7W1hRz_AYMPY5l6jobgDfkWYVIi" style="width:600;height:350px"/></center><br>

&emsp;&emsp;&emsp;&emsp;<b> - Подход на основе rendering (обработки и визуализации) в 3D фреймворке, таким образом генерируются синтетические изображения с разметкой. <br>&emsp;&emsp;&emsp;&emsp;&emsp;Один из таких инструментов - [Blender](https://www.blender.org/). Удобен тем что: open source, позволяет строить pipeline на Python и возможен запуск синтеза изображений без GUI.</b><br>
<center><img src="https://drive.google.com/uc?id=1_h-dgR_zQR9Glsjz2js620NIlQ6trm8E" style="width:720;height:378px"/></center>

#### <b> По теме синтез изображений на Blender рассмотрим на примере модели автомобиля с номером:</b>
    4.1. Что представляет из себя Blender и из каких основных компонент состоит
    4.2. Настройка сцены для задачи детекции (автомобиля и его номера)
    4.3. Работа с Blender compositing для решаемой задачи
    4.4. Python API и его применение для синтеза изображений
    4.5. Финальная настройка pipeline синтеза изображений для запуска в режиме GUI less

#### <b> <center>4.1.  Blender и его основные компоненты</center></b>
#####<b> <left>4.1.1  Подготовка к работе</left></b>
Для работы нам понадобиться [проект Blender с моделями](https://drive.google.com/file/d/1YwSfkr0Ec6pl1etq-2k27q28czKBeVkv/view?usp=share_link), Который необходимо скачать и запустить.
#####<b> <left>4.1.2  Работа в Blender проекте</left></b>
&emsp;&emsp;Blender - бесплатное open source приложение с набором инструментов по построению 3D pipeline для: моделирования, анимации, симуляции, рендерингу (процесс получения изображения из 3D сцены) изображений, отслеживания движения, редактирования видео и создания игр.<br>
&emsp;&emsp;В его состав входит достаточно большое количество инструментов, среди них можно выделить основные необходимы для синтеза изображений: <font color="purple">Layout, Compositing и Scripting</font><br>
<center><img src="https://drive.google.com/uc?id=1_FwgTSzwtNcsM7aopwNNFqYfSGAMdIdQ" style="width:1280;height:720px"/></center>
<center><font color="purple"><b>Layout</b></font> - инструментарий для моделирования и настройки элементов 3D сцены</center><br>
<center><img src="https://drive.google.com/uc?id=1ortjhQLKE91FK6KmEk1kMF1hBsUeSneE" style="width:1280;height:720px"/></center>
<center><font color="purple"><b>Compositing</b></font> - инструментарий для финализации изображения (получение отрендеренного изображения, получения маски alpha канала, сохранения итогового изображения и т.д.).</center><br>
<center><img src="https://drive.google.com/uc?id=17uPfiwutWihxznp-skJ4JGJk0X2PgYsV" style="width:1280;height:720px"/></center>
<center><font color="purple"><b>Scripting</b></font> - инструментарий Python API для автоматизации рендеринга и синтеза изображений</center>

In [ ]:
from ipywidgets import Video
video = Video.from_file("/content/data/materials/video/layout_compositing_scripting.mp4")
video

[<center>Демонстрация работы с инструментарием Layout, Compositing и Scripting</center>](https://drive.google.com/file/d/1ccJaGnKIgHP1ByRC5IBUEY9zNbWWKfeS/view?usp=share_link)

### <b><center>Layout</center></b>
&emsp;&emsp;Основной инструментарий работы с 3D сценой представлен на панелях:<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; - Панель перемещения, вращения, масштабирования (Transform - справа)<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; - Панель управления коллекциями элементов 3D сцены и слоями(Scene Collection - справа)<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; - Панель настроек свойств элементов 3D сцены(панель с иконками справа снизу)<br>
&emsp;&emsp;Для нормального рендеринга в 3D сцене должны присутствовать:<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;1) любые 3D объекты сцены (машина, стены, пол, деревья, здания и т.д.);<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;2) освещение (в blender это источники света: солнечный, точечный, точечный направленный и точечный площадной);<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;3) камеры (отвечают за ракурс визуализации)<br>
&emsp;&emsp;Каждый объект сцены может быть перемещен, повернут или отмасштабирован, а также обладает специализированными свойствами (например источник света имеет силу света - strength).<br>
&emsp;&emsp;Объекты сцены можно также группировать в коллекции - collection и выполнять операции над такой группой. 
<br>&emsp;&emsp;Для изоляции объектов рендеринга можно создавать отдельные слои - layout (справа сверху)

In [ ]:
from ipywidgets import Video
video = Video.from_file("/content/data/materials/video/layout.mp4")
video

[<center>Layout инструментарий. Панели трансформации  и работа с объектами, их параметрами и создание коллекций </center>](https://drive.google.com/file/d/1KHxBcBGUCZ640dXOXwREK8XpEBMH7Iam/view?usp=share_link)

#### <b> <center>4.2.  Настройка сцены для решения задачи детекции</center></b>

&emsp;&emsp;Для создания рендеринга с разметкой для задачи детекции необходимо:<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1. Создать коллекции для каждого класса (Car, License plate)<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2. Создать слой под каждый отдельный класс (All, Car_layer, License_plate_layer)<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 3. В каждом слое изолировать коллекции, которые не должны участвовать в рендеринге<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 4. Настройка параметров рендера<br>
&emsp;&emsp; Создадим слои: All, Car_layer, License_plate_layer и изолируем коллекции, изолировав все коллекции кроме класса слоя <br>

##### <b> <center>1. Создание коллекции под каждый отдельный класс</center></b><br>
Создадим коллекции для генерации разметки под каждый класс: Car, License plate, Background

In [ ]:
from ipywidgets import Video
video = Video.from_file("/content/data/materials/video/create_collections.mp4")
video

[<center>Создание коллекции под каждый отдельный класс</center>](https://drive.google.com/file/d/1E8lOOcguXcn4g5ONq_9dEd7Vp6k6bF6r/view?usp=share_link)

##### <b> <center>2. Создание слоев под каждый отдельный класс и изоляция коллекций</center></b><br>
Создадим слои (Car_layer, License_plate_layer) и изолируем классы в каждом слое рендеринга. Это необходимо для рендеринга классов объектов по отдельности.

In [ ]:
from ipywidgets import Video
video = Video.from_file("/content/data/materials/video/create_layers_isolation_object_classes.mp4")
video

[<center>Создание слоев под каждый отдельный класс и изоляция коллекций</center>](https://drive.google.com/file/d/1G_L7g0xNTGmEaplRID2a_41dz2ht-2Mp/view?usp=share_link)

##### <b> <center>3. Настройка параметров рендера</center></b><br>
Для финальной настройки рендера необходимо установить параметр прозрачности фона:

In [ ]:
from ipywidgets import Video
video = Video.from_file("/content/data/materials/video/render_setup.mp4")
video

[<center>Настройка параметров рендера</center>](https://drive.google.com/file/d/1--Ir-eGolkLtaQuE477OViKnZ3P-sLhw/view?usp=share_link)

Если нажать F12, то запустится рендеринг и можно увидеть, как проходит рендеринг по каждому слою (классу объектов)

#### <b> <center>4.3. Работа с Blender Compositing для решения задачи разметки</center></b>
&emsp;&emsp;Инструментарий в виде графа, где происходит постобработка отрендеренного изображения. Граф позволяет менять цвета, работать с альфа-каналом, смешивать изображения, получая финализированное изображение. Помимо работы с изображение, имеется возможность также сохранять промежуточные изображения и их визуализация.<br>&emsp;&emsp;Весь граф представлен узлами - операторами над изображениями, и ребрами - связями между узлами. Связь узлов показывает как и куда изображение из левого узла передается в правый<br>

In [ ]:
from ipywidgets import Video
video = Video.from_file("/content/data/materials/video/compositing.mp4")
video

[<center>Работа с Blender Compositing для решения задачи разметки</center>](https://drive.google.com/file/d/1gryaBxFdV5rtJdIOWSXja128anqx7rcU/view?usp=share_link)

&emsp;&emsp;Постобработка для изображений под разметку представляет граф в состав которого входят:<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1. Входные узлы - Render Layers - позволяют получить изображения со слоев рендеринга<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2. Промежуточные - Alpha Over - создают бинарную маску по альфа каналу<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 3. Выходные - Viewer - визуализируют в blender compositing<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 4. Выходные - File Output - сохраняют финальное изображение<br>
&emsp;&emsp; Создадим простой граф с сохранением готового изображения в папке: ./data/materials/blender_scene/dataset/images <br>
(F12 - для запуска рендеринга и прогона изображения через граф compositing)

In [ ]:
from ipywidgets import Video
video = Video.from_file("/content/data/materials/video/compositing_image_synthesis.mp4")
video

[<center>Создание графа для генерации изображения</center>](https://drive.google.com/file/d/1gryaBxFdV5rtJdIOWSXja128anqx7rcU/view?usp=share_link)

&emsp;&emsp; Создадим еще один граф с получением битовой маски автомобиля и сохранение изображения в папке: /content/data/materials/blender_scene/dataset/masks/car <br>(Для перемещение графа можно зажать на любом узле колесико мыши и перетащить, а приблизить/отдалить колесиком мыши вперед/назад)

In [ ]:
from ipywidgets import Video
video = Video.from_file("/content/data/materials/video/creating_bitmask_car_graph.mp4")
video

[<center>Создание графа для генерации битовой маски</center>](https://drive.google.com/file/d/1G6M84u7Tu3iuZfhk7scZ893mH461eNG6/view?usp=share_link)

<b><i><font color="purple">&emsp;&emsp;Задание:<br> 
        &emsp;&emsp;&emsp;&emsp;1. Создать граф с получением битовой маски номера автомобиля<br>
</font></i></b>

#### <b> <center>4.4. Python API и его применение для синтеза изображений</center></b>
Scripting - инструментарий с Python API. Состоит из нескольких панелей: <br>&emsp;&emsp;&emsp;&emsp;- Layout - для просмотра и редактирования 3D сцены <br>&emsp;&emsp;&emsp;&emsp;-  Terminal python console - для быстрой проверки и тестирования кода<br>&emsp;&emsp;&emsp;&emsp;- Information панель - для отображения сообщений по действиям в Blender (очень полезно для автоматизации). В режиме отладки показывает сообщения в виде кода по любому действию в Blender.<br>&emsp;&emsp;&emsp;&emsp;- Code панель - для написания и запуска кода<br>

In [ ]:
from ipywidgets import Video
video = Video.from_file("/content/data/materials/video/scripting.mp4")
video

[<center> Scripting панель</center>](https://drive.google.com/file/d/1q0ZQx_2Hl70IGjN61hJVanhwVmgWVUpX/view?usp=share_link)

Напишем скрипт перехода в режим отладки, запустим и потестируем этот режим на примере автоматизации создания куба в 3D Scene. <br>(Запуск скриптов в Blender - Alt+P или кнопка play в меню code панели)

In [ ]:
from ipywidgets import Video
video = Video.from_file("/content/data/materials/video/scripting_debug_mode.mp4")
video

[<center> Режим Debug</center>](https://drive.google.com/file/d/1HZVH6gtrtaOA2jTZsIxEdvbYvhQElTKf/view?usp=share_link)

&emsp;&emsp;Основная идея автоматизации синтеза изображений в Blender - написание скрипта, который рендерит изображения с максимальной вариативностью и соответствующей разметкой. Для получения таких изображений необходимо выставить камеру (отвечающую за ракурс) в случайном месте с фокусом на объекте(ах) для детекции (сегментации и т.д.) и по необходимости изменить внешний вид объектов, их положение и освещение.<br>&emsp;&emsp;Проверим допустимые границы видимости автомобиля через движение и поворот камерой.<br>(Для скрытия/показа панели в Layout можно использовать кнопку N)

In [ ]:
from ipywidgets import Video
video = Video.from_file("/content/data/materials/video/camera_variation_definition.mp4")
video

[<center> Идея автоматизации синтеза изображения</center>](https://drive.google.com/file/d/1cJrpsPm1D5ywOWOUNiyKMxujOY6IM_kU/view?usp=share_link)

#### <b> <center>4.5. Финальная настройка pipeline синтеза изображений</center></b>
Напишем скрипт для синтеза изображений.<br>
<b><i><font color="purple">&emsp;&emsp;Задание:<br> 
        &emsp;&emsp;&emsp;&emsp;1. Дописать функцию перемещения камеры (move_camera) для вариативного синтеза изображений<br>
</font></i></b>

In [ ]:
%%writefile /content/data/materials/blender_scene/synthesis_images.py
import bpy
import numpy as np
import time
import math
from os import listdir, mkdir, getcwd
import random
import uuid
import argparse

# README https://docs.blender.org/manual/en/2.79/data_system/files/relative_paths.html

# Получение имени изображения 
# и файлов разметки(должно быть одинаковым)
def get_name():
    return f"{str(uuid.uuid4()).replace('-','')}_frame_"

# Выставление одинакового имени файлов для всех
# узлов File Output в Compositing
def set_name_to_output_file(filename):
    # set own filename for each image file
    file_output_nodes = find_nodes_byname("File Output")
    for file_output_node in file_output_nodes:
        file_output_node.file_slots[0].path = filename

# Поиск узла в Compositing по имени
def find_nodes_byname(node_name:str) -> list:
    result_nodes = []
    bpy.context.scene.use_nodes = True 
    tree = bpy.context.scene.node_tree.nodes
    image_node = None
    for node in tree:
        if node.bl_label == node_name:
            result_nodes.append(node)
    return result_nodes

# Функция рендеринга. Выставляет параметры рендеринга и
# производит рендеринг с прогонкой через граф Compositing
# и сохранением всех файлов изображений
def render(**kwargs):
    # Получение объектов сцены и рендера
    render = bpy.context.scene.render
    scene = bpy.context.scene
    # Параметры сглаживания шума создаваемого рендером
    scene.cycles.use_denoising = True
    # scene.cycles.denoiser = 'OPTIX' # README https://github.com/dfelinto/blender/blob/master/intern/cycles/blender/addon/properties.py
    scene.cycles.preview_denoising_input_passes = 'RGB_ALBEDO'
    # Качество рендеринга - чем выше тем дольше рендер одного изображений и выше качество
    scene.cycles.samples = kwargs['quality'] if 'quality' in kwargs else 32



    # Имя и путь к файлу изображения генерируемого по умолчанию
    render.filepath = f'{getcwd()}/test_image'
    # Рендер CYCLES - более реалистичное качество чем Evee
    render.engine = 'CYCLES'
    # Параметры для работы с CUDA
    scene.cycles.device = 'GPU'
    # Разрешение изображений
    render.resolution_x = 1920
    render.resolution_y = 1080
    # Параметр прозрачности при рендере фона, для png формата и работой с альфа каналом в Compositing
    render.film_transparent = True
    # Создание нового имени файла 
    set_name_to_output_file(get_name())
    
    # Запуск рендеринга
    if 'layer' in kwargs:
        bpy.ops.render.render(write_still=True, layer=kwargs['layer'])
    else:
        bpy.ops.render.render(write_still=True)
    print('Rendering has completed')

# Перемещение и вращение камеры на случайное расстояние и угол
def move_camera():
    # Перемещение камеры по оси X
    bpy.data.objects["Camera"].location[0] = bpy.data.objects["Camera"].location[0] + random.randint(0,100)
    # Перемещение камеры по оси Y
    bpy.data.objects["Camera"].location[1] = bpy.data.objects["Camera"].location[1] + random.randint(0,100)
    # Перемещение камеры по оси Z
    bpy.data.objects["Camera"].location[2] = bpy.data.objects["Camera"].location[2] + random.randint(0,100)
    # Поворот камеры вокруг оси X
    bpy.data.objects["Camera"].rotation_euler[0] = 2 * math.pi * random.randint(-45,-30) / 360
    # Поворот камеры вокруг оси Z
    bpy.data.objects["Camera"].rotation_euler[2] = 2 * math.pi * random.randint(-45,-30) / 360

if __name__ == "__main__":
    images_count = 3
    for index in range(images_count):
        print(f"====== Start synthesis image number {index + 1} =====")
        move_camera()
        render(**{'quality':10})


Blender API Python позволяет запускать рендер (синтез) изображений из командной строки с необходимой сценой и отдельным python скриптом не загружая GUI Blender. Для этого необходимо передать в команду blender параметры:  путь к blender сцене, background - для запуска в NoGUI режиме и параметра python с указанием пути к python скрипту.<br>Пример запуска в терминале ниже: 

In [ ]:
!blender /content/data/materials/blender_scene/synthesis_scene_ready.blend --background --python /content/data/materials/blender_scene/synthesis_images_ready.py

После прогона скрипта в папке /content/data/materials/blender_scene/dataset/ в папках images и masks синтезируются файлы изображений с объектами и их масками.<br><br>Пример функции получения координат boundingbox по синтезированной маске:

In [ ]:
import cv2
import numpy as np
import ipyplot
########## Constants ##################
color = (255, 0, 255) # фиолетовый цвет - boundingbox изображения
thickness = 1 # толщина линии
path_to_mask = "/content/data/materials/blender_scene/dataset/masks/car/2c62c2414cff47dbb0e7df0a118f05d9_frame_0001.png" # Путь к маске изображения автомобиля
#######################################

# Функция получения координат boundingbox из маски изображения
# path_to_mask - путь к маске-изображению
# mask_threshold - порог отсечения шума вызванного особенностями рендеринга
# visualize - параметр отображения битовой маски и boundingbox после обработки
#  color - параметр цвета boundingbox
#  thickness - параметр толщины boundingbox
def get_bbox(path_to_mask: str, mask_threshold: int=10, visualize: bool=True, color: tuple=(255, 0, 255), thickness: int=1)->None:
        image = cv2.imread(path_to_mask)
        # Получение bitmask
        bitmask =  image[:, :, 0].copy()
        bitmask[np.where(bitmask <= mask_threshold)] = 0
        bitmask[np.where(bitmask > mask_threshold)] = 1
        # Получение координат boundingbox
        points =  np.where(bitmask == 1)
        xmin = int(np.min(points[1]))
        xmax = int(np.max(points[1]))
        ymin = int(np.min(points[0]))
        ymax = int(np.max(points[0]))
        # Визуализация
        if visualize:
            _ , width, _ = image.shape
            image[np.where(image <= mask_threshold)] = 0
            image[np.where(image > mask_threshold)] = 255
            # Отображение полученного boundingbox
            image = cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color=color, thickness=thickness)
            ipyplot.plot_images([image], max_images=1, img_width = width//2)
        return (xmin, ymin, xmax, ymax)

get_bbox(path_to_mask)

<b><i><font color="green">Задание к самостоятельной проработке:<br> 
        &emsp;&emsp;&emsp;&emsp;1. Написать функцию получения разметки из всех классов синтезированных в Blender <br>
</font></i></b><br>